In [4]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame([76_212_168, 92_228_496, 106_021_537, 123_202_624, 132_164_569, 151_325_798, 179_323_175, 203_302_031, 226_542_199])
print(df)

           0
0   76212168
1   92228496
2  106021537
3  123202624
4  132164569
5  151325798
6  179323175
7  203302031
8  226542199
